<a href="https://colab.research.google.com/github/Xiaoyan-Adele/Automated-Hate-Speech-Detection/blob/master/Codes/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setting up

In [0]:
!python -m spacy download en_core_web_lg


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 

import pandas as pd
import io
import unicodedata
import numpy as np
import re
import string



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#2. Load the corpus file form Google Drive into Colab


In [0]:
# Upload from local
from google.colab import files
uploaded = files.upload()
print("len(uploaded.keys():", len(uploaded.keys()))

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

# Get dataframe of the tweets in the labeled dataset 
df_tweets = pd.read_csv('labeled_data.csv',encoding = 'utf-8',sep=',',index_col=0)

#selecting the right column for data preprocessing 
tweets_pd_raw = df_tweets.iloc[:,5]

#inspecting the selected column
tweets_pd_raw.head(5)

len(uploaded.keys(): 0


0    !!! RT @mayasolovely: As a woman you shouldn't...
1    !!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2    !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3    !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4    !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
Name: tweet, dtype: object

In [0]:
tweets_pd = pd.read_csv('labeled_data.csv',encoding = 'utf-8',sep=',',index_col=0)

#inspecting the selected column
tweets_pd.head(5)

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


# 3. Preprocessing: replace, tokenize sentences and words, remove stopwords, use stemmer & lemmatizer



In [0]:
# Load the regular expression library
import re
import spacy
import string
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
from nltk.corpus import stopwords
stop = stopwords.words('english')

import spacy
nlp = spacy.load('/usr/local/lib/python3.6/dist-packages/en_core_web_lg/en_core_web_lg-2.2.5')


##remove mentions
mention_regex = '@[\w\-]+'
tweets_pd['tweet'] = tweets_pd['tweet'].map(lambda x: re.sub(mention_regex, ' ', x))

# Remove punctuation
tweets_pd['tweet'] = tweets_pd['tweet'].map(lambda x: re.sub('[,-:\.!?]', '', x))

#remove and \n
tweets_pd['tweet'] = tweets_pd['tweet'].map(lambda x: re.sub('\n','', x))

#remove numbers, excessive white space
space_pattern = '\s+'
tweets_pd['tweet'] = tweets_pd['tweet'].map(lambda x: re.sub(space_pattern,' ', x))
tweets_pd['tweet'] = tweets_pd['tweet'].map(lambda x: re.sub('[0-9]','', x))

# Convert the titles to lowercase
tweets_pd['tweet'] = tweets_pd['tweet'].map(lambda x: x.lower().split())

# Do lemmatization keeping only noun, adj, vb, adv
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


tweets_pd['tweet'] = lemmatization(tweets_pd['tweet'], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#remove stop words
def remove_stopwords(X_list):
    filtered_list =[]
    for it in X_list:
        temp = []
        for word in it:
            if(word not in stop):
                temp.append(word)
        filtered_list.append(temp)
        temp=[] 
    return filtered_list

tweets_pd['tweet'] = remove_stopwords(tweets_pd['tweet'])

#take out numbers and 
tweets_pd.head(10)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,"[woman, complain, clean, house, man, always, t..."
1,3,0,3,0,1,"[boy, bad, hoe, place]"
2,3,0,3,0,1,"[ever, fuck, bitch, start, cry, confuse, shit]"
3,3,0,2,1,1,"[look, tranny]"
4,6,0,6,0,1,"[shit, hear, may, true, may, faker, bitch, tel..."
5,3,1,2,0,1,"[shit, blow, meclaim, faithful, still, fuck, h..."
6,3,0,3,0,1,"[sit, hate, bitch, get, much, shit, go]"
7,3,0,3,0,1,"[tired, big, bitch, come, skinny, girl, #]"
8,3,0,3,0,1,"[may, bitch]"
9,3,1,2,0,1,"[hobby, include, fight]"


In [0]:
print(type(tweets_pd))

<class 'pandas.core.frame.DataFrame'>


# 4. Constructing Naive Bayer Model






In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics


# Split dataset into training set and test set. 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(tweets_pd['tweet'], tweets_pd['class'], test_size=0.3,random_state=100) 



X_train = [' '.join(token) for token in X_train]
X_test = [' '.join(token) for token in X_test]


tf_idf = Pipeline([('cv',CountVectorizer()), ('tfidf_transformer',TfidfTransformer(smooth_idf=True,use_idf=True))])

X_train = tf_idf.fit_transform(X_train).todense()
X_test = tf_idf.transform(X_test).todense()


clf = MultinomialNB().fit(X_train, y_train)
y_score = clf.predict(X_test)



In [0]:
label = y_test.values
acc = (y_score == label).mean()
print('acc: {:.5f}'.format(acc))

acc: 0.79220
